# Calculate ILAMB Data
Creates annual, regridded data as well as month of max and climatology, from raw ILAMB data. Uses a config file (e.g. ILAMB_obs.cfg) to get information about where ILAMB data is located and what the files, models, and variable names are.

In [1]:
import os
import warnings
import xarray as xr

import fates_calibration_library.utils as utils
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.analysis_functions as analysis

# ignore warnings about latitude being outside of -90, 90
warnings.simplefilter("ignore", UserWarning)

In [2]:
# config file for ILAMB data
ilamb_cfg = '/glade/work/afoster/FATES_calibration/scripts/configs/ILAMB_obs.yaml'

# land area file
land_area_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")

In [3]:
# config dictionary for running functions
config_dict = {
    'top_dir': '/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA',  # location of raw ILAMB data
    'out_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs',  # where to put compiled ILAMB data
    'start_date': '2000-01-01',  # start date to filter to
    'end_date': '2014-12-31',  # end date to filter to
    'user': 'afoster@ucar.edu',  # user
    'clobber': False,  # overwrite files?
}

In [4]:
# read in the ILAMB data dictionary
ilamb_dict = utils.get_config_file(ilamb_cfg)

In [5]:
# create a target grid to regrid to
target_grid = analysis.create_target_grid(land_area_ds, 'FSR')

In [ ]:
dataset = 'ESACCI_BIOMASS'
attributes = ilamb_dict[dataset]

In [ ]:
import importlib
import fates_calibration_library.ilamb_functions
import fates_calibration_library.analysis_functions
importlib.reload(fates_calibration_library.ilamb_functions)
importlib.reload(fates_calibration_library.analysis_functions)
import numpy as np

In [ ]:
# read or compute ILAMB data
ilamb_dat, original_file = ilamb.read_ilamb_data(
    config_dict, ilamb_dict, attributes
)

In [ ]:
# prepare metadata
metadata = {
    "units": attributes["units"],
    "longname": attributes["longname"],
    "original_file": original_file,
    "user": config_dict["user"],
}

In [ ]:
annual = analysis.calculate_annual_mean(ilamb_dat[attributes["in_var"]],
                                        utils.evaluate_conversion_factor(attributes["time_conversion_factor"]))

In [ ]:
annual.plot()

In [ ]:
# calculate annual dataset
annual_ds = ilamb.get_annual_ds(
    ilamb_dat,
    attributes["in_var"],
    attributes["out_var"],
    utils.evaluate_conversion_factor(attributes["time_conversion_factor"]),
    metadata,
)
regridded_annual = ilamb.regrid_ilamb_ds(annual_ds, target_grid, attributes["out_var"])
if "year" in regridded_annual[attributes['out_var']].dims:
    sum_annual = regridded_annual[attributes['out_var']].sum(dim="year")
else:
    sum_annual = regridded_annual[attributes['out_var']] 
regridded_annual = regridded_annual.where(np.abs(sum_annual) > 0.0)

In [ ]:
# monthly mean
monthly_mean = analysis.calculate_monthly_mean(
    ilamb_dat[attributes["in_var"]],
    utils.evaluate_conversion_factor(attributes["time_conversion_factor"]),
)
# regrided monthly mean
regridded_monthly = ilamb.regrid_ilamb_ds(
    monthly_mean.to_dataset(name=f"{attributes['out_var']}_monthly"),
    target_grid,
    f"{attributes['out_var']}_monthly")
sum_monthly = regridded_monthly[f"{attributes['out_var']}_monthly"].sum(dim='month')
regridded_monthly = regridded_monthly.where(np.abs(sum_monthly) > 0.0)

In [ ]:
# calculate month of maximum value
month_of_max = analysis.get_monthly_max(regridded_monthly[f"{attributes['out_var']}_monthly"]).to_dataset(
    name=f"{attributes['out_var']}_month_of_max"
)

In [ ]:
month_of_max[f"{attributes['out_var']}_month_of_max"].plot(cmap='jet')

In [ ]:
utils.evaluate_conversion_factor(attributes["area_conversion_factor"]) is None

In [ ]:
climatology_ds = ilamb.get_ilamb_climatology(
    regridded_monthly,
    attributes['out_var'],
    utils.evaluate_conversion_factor(attributes["area_conversion_factor"])
)

In [ ]:
climatology_ds[f"{attributes['out_var']}_cycle"].plot()

In [6]:
# loop through dictionary and process the ILAMB observations
# script will skip files that already exist unless clobber: True
ilamb.get_all_ilamb_data(config_dict, ilamb_dict, target_grid)

File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_GPP.nc for FLUXCOM_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/WECANN_GPP.nc for WECANN_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/GBAF_GPP.nc for GBAF_GPP exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_NEE.nc for FLUXCOM_NEE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/AVHRR_LAI.nc for AVHRR_LAI exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/AVH15C1_LAI.nc for AVH15C1_LAI exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/FLUXCOM_LE.nc for FLUXCOM_LE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/CLASS_LE.nc for CLASS_LE exists, skipping
File /glade/work/afoster/FATES_calibration/observations/ILAMB_obs/WECANN_LE.nc for WECANN_LE exists, skippin

In [ ]:
# compile data into one dataset
compiled_ds = ilamb.compile_ilamb_datasets(config_dict['out_dir'], ilamb_dict,
                                          target_grid.area)
compiled_ds.to_netcdf(os.path.join('/glade/work/afoster/FATES_calibration',
                                  'observations/all_ILAMB_obs.nc'), mode="w")